In [1]:
from Bio.PDB import PDBList
import os
import requests
import pandas as pd
import subprocess
import time
import tempfile

import asyncio
import httpx
import nest_asyncio

import duckdb as db
import numpy as np

In [2]:
df_test = pd.read_csv('./pair_sample.csv', index_col=0)
df_test

,meso_pid,thermo_pid,meso_pdb,thermo_pdb
0,P9WJA3,A0A1M6N9Z6,1Y5H,NaN
1,I6XFS7,A0A1M6WSV2,6M1C,NaN
2,Q65EQ1,A0A521F3Z2,6NKG,NaN
3,F5HRS7,A0A2T0LBQ2,7QH4,NaN
4,P9WHM1,C7MUW2,3LP6,NaN
...,...,...,...,...
95,A0A4Z0GXN3,A0A1G7W5M9,NaN,NaN
96,G4H893,A0A1W6VMF1,NaN,NaN
97,A0A120GMI5,Q5L0I9,NaN,NaN
98,A0A4V2YRI4,A0A3N2GW27,NaN,NaN


In [17]:
async def download_aff(session, url, filename):
    try:
        response = await session.get(url)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded file: {filename}")
            return True
        else:
            print(f"Failed to download file: {filename}. Status code: {response.status_code}")
            return False
    except httpx.RequestError as e:
        print(f"Error while downloading file: {filename}. Exception: {str(e)}")
        return False

async def download_af(row, u_column, pdb_dir):
    uniprot_id = getattr(row, u_column)
    url = f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb'
    filename = f'{pdb_dir}/{uniprot_id}.pdb'

    async with httpx.AsyncClient() as client:
        success = await download_aff(client, url, filename)
        return success

def run_download_af_all(df, pdb_column, u_column, pdb_dir):
    nest_asyncio.apply()

    async def download_af_all():
        tasks = []
        success_count = 0

        if not os.path.exists(pdb_dir):
            os.makedirs(pdb_dir)

        for row in df.itertuples(index=False):
            if pd.isna(getattr(row, pdb_column)):
                task = asyncio.create_task(download_af(row, u_column, pdb_dir))
                tasks.append(task)

        results = await asyncio.gather(*tasks)
        success_count = sum(results)

        print(f"Successfully downloaded {success_count} files out of {len(df)}")

    asyncio.run(download_af_all())

def download_pdb(df, pdb_column, pdb_dir):
    pdbl = PDBList()
    for i, row in df.iterrows():
        pdb_id = row[pdb_column]
        if not pd.isna(pdb_id):  # check for NaN value in PDB IDs column
            pdbl.retrieve_pdb_file(pdb_id, pdir=pdb_dir, file_format='pdb')
            file_path = os.path.join(pdb_dir, f'pdb{pdb_id.lower()}.ent')
            if os.path.exists(file_path):
                os.rename(file_path, os.path.join(pdb_dir, f'{pdb_id}.pdb'))
            else:
                pass

def download_structure(df, pdb_column, u_column, pdb_dir):
    start_time = time.time()  # Start measuring time
    if not os.path.exists(pdb_dir):
        os.makedirs(pdb_dir)
    download_pdb(df, pdb_column, pdb_dir)    
    run_download_af_all(df, pdb_column, u_column, pdb_dir)
    end_time = time.time()  # Stop measuring time
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")
    pass

In [5]:
df_small = df_test.sample(3)

In [19]:
df_small

,meso_pid,thermo_pid,meso_pdb,thermo_pdb,p_value
34,Q9I5C9,A0A840UQH5,3UMC,NaN,2
10,A0A1H2D2H5,A0A3N2H419,7OII,NaN,2
48,Q8DNC2,A0A1W6VND0,4ZMS,NaN,2


In [18]:
download_structure(df_small, 'meso_pdb','meso_pid', 'af')

Desired structure doesn't exist
Successfully downloaded 0 files out of 3
Execution time: 1.3063068389892578 seconds


In [20]:
download_structure(df_small, 'thermo_pdb','thermo_pid', 'af')

Downloaded file: af/A0A3N2H419.pdb
Downloaded file: af/A0A1W6VND0.pdb
Downloaded file: af/A0A840UQH5.pdb
Successfully downloaded 3 files out of 3
Execution time: 0.2657814025878906 seconds


In [30]:
def run_fatcat(df, pdb_dir):
    p_values = []  # List to store the extracted p-values

    for index, row in df.iterrows():
        if not pd.isna(row['meso_pdb']):
            p1 = row['meso_pdb']
        else:
            p1 = row['meso_pid']
        
        if not pd.isna(row['thermo_pdb']):
            p2 = row['thermo_pdb']
        else:
            p2 = row['thermo_pid']
        
        # Check if the structure files exist in the 'checking' folder
        p1_file = f'{p1}.pdb'
        p2_file = f'{p2}.pdb'
        if not os.path.exists(os.path.join(pdb_dir, p1_file)) or not os.path.exists(os.path.join(pdb_dir, p2_file)):
            # Assign NaN as the p-value instead of dropping the row
            p_values.append(np.nan)
            continue

        # Set the FATCAT command and its arguments
        cmd = ['FATCAT', '-p1', p1_file, '-p2', p2_file, '-i', pdb_dir, '-q']
        
        # Run the FATCAT command and capture the output
        result = subprocess.run(cmd, capture_output=True, text=True)
        output = result.stdout

        # Find the line containing the p-value
        p_value_line = next(line for line in output.split('\n') if line.startswith("P-value"))

        # Extract the p-value and convert it to numeric value
        p_value = float(p_value_line.split()[1])
        
        # Check if p-value is less than 0.05 and assign 1 or 0 accordingly
        if p_value < 0.05:
            p_values.append(str(1))
        else:
            p_values.append(str(0))

    df.loc[:, 'p_value'] = p_values  # Use .loc to set the 'p_value' column
    return df

In [ ]:
conn = db.connect("./pairpro_50k.db")
df = conn.execute("""SELECT thermo_pid, meso_pid FROM pairpro.final""").df()

download_structure(df, 'thermo_pdb','thermo_pid', 'af')
download_structure(df, 'meso_pdb','meso_pid', 'af')
run_fatcat(df, 'af')

In [31]:
df_result = run_fatcat(df_small, pdb_dir='af')

In [32]:
df_result

,meso_pid,thermo_pid,meso_pdb,thermo_pdb,p_value
34,Q9I5C9,A0A840UQH5,3UMC,NaN,1
10,A0A1H2D2H5,A0A3N2H419,7OII,NaN,NaN
48,Q8DNC2,A0A1W6VND0,4ZMS,NaN,1


In [ ]:
# Quick fatcat command: 
# FATCATQue.pl timeused pair.list -q >pair.aln (only work when files are in the same directory)
# FATCAT -p1 A0JNW5.pdb -p2 A1L1K8.pdb -o A0JNW5_A1L1K8 -m
# FATCAT -p1 P0A9P0.pdb -p2 Q9H3D4.pdb -i ./checking -o test -m (-i directory of PDB files), result file is in ./

#FATCAT command help: 
FATCAT <-p1 file> <-p2 file> (the input pdb files)
  [-o output-initial] (default tmp)
  [-i string] (data directory for both structures, default ./)
  [-i1 string] (data directory for 1st structures, default ./)
  [-i2 string] (data directory for 2st structures, default ./)
  [-s1 num] (read start position of protein 1, default from the begin)
  [-s2 num] (read start position of protein 2, default from the begin)
  [-l1 num] (read length of protein 1, default whole protein)
  [-l2 num] (read length of protein 2, default whole protein)
  [-r] (force program run rigid structural alignment, default off)
  [-filter probcut] (filter the alignment quickly, set a big probcut, eg 0.2, useful in database searching, default off)
  [-sparse number[0-3]] (sparsely fragment sampling, for speeding up the calculation, default off)
  [-b] (print a basic report to stdout)
  [-f] (print a full report to stdout. When -b or -f is on, following options are all automatically off)
  [-m] (print alignment to a file)
  [-q] (print alignment to stdout, useful in database-search in queue)
  [-ab] (print the postscript graph of all AFPs and final AFP chain in black-white to a file)
  [-ac] (print the postscript graph of all AFPs and final AFP chain in color to a file)
  [-c] (print AFP chaining result to file.chain.txt)
  [-t] (print the files of transformed pdbs and corresponding rasmol scripts)
  [-s] (print the files of superimposed pdbs and corresponding rasmol scripts)
  [-time] (print the total running time, default off)

In [24]:
def run_fatcat(df, pdb_dir):
    p_values = []  # List to store the extracted p-values
    rows_to_drop = []  # List to store the indices of rows to be dropped

    for index, row in df.iterrows():
        if not pd.isna(row['meso_pdb']):
            p1 = row['meso_pdb']
        else:
            p1 = row['meso_pid']
        
        if not pd.isna(row['thermo_pdb']):
            p2 = row['thermo_pdb']
        else:
            p2 = row['thermo_pid']
        
        # Check if the structure files exist in the 'checking' folder
        p1_file = f'{p1}.pdb'
        p2_file = f'{p2}.pdb'
        if not os.path.exists(os.path.join(pdb_dir, p1_file)) or not os.path.exists(os.path.join(pdb_dir, p2_file)):
            # Append the index of the row to the list of rows to be dropped
            rows_to_drop.append(index)
            continue

        # Set the FATCAT command and its arguments
        cmd = ['FATCAT', '-p1', p1_file, '-p2', p2_file, '-i', pdb_dir, '-q']
        
        # Run the FATCAT command and capture the output
        result = subprocess.run(cmd, capture_output=True, text=True)
        output = result.stdout

        # Find the line containing the p-value
        p_value_line = next(line for line in output.split('\n') if line.startswith("P-value"))

        # Extract the p-value
        p_value = float(p_value_line.split()[1])
        
        # Append the p-value to the list
        p_values.append(p_value)

    # Drop the rows with missing structure files from the dataframe
    df = df.drop(rows_to_drop)
    
    df.loc[:, 'p_value'] = p_values  # Use .loc to set the 'p_value' column

    cmd_2 = """CREATE OR REPLACE TEMP TABLE tmp
                AS SELECT * FROM df"""
    cmd_3 = """ALTER TABLE pairpro.final ADD COLUMN p_value FLOAT"""
    cmd_4 = """UPDATE pairpro.final SET p_value = tmp.p_value
                FROM pairpro.final
                INNER JOIN tmp ON (pairpro.final.meso_pid = pairpro.tmp.meso_pid)
                AND (pairpro.final.thermo_pid = pairpro.tmp.thermo_pid)"""
    conn.sql(cmd_2)
    conn.execute(cmd_3)
    conn.execute(cmd_4)
    conn.commit()

In [ ]:
conn = db.connect("./pairpro_50k.db")
con_df = conn.execute("""SELECT * FROM pairpro.final LIMIT 100""").fetch_df()

con_df[['meso_pdb', 'thermo_pdb']]= np.nan
con_df